<img align="right" src = "https://raw.githubusercontent.com/arsilva87/statsbook/main/figs/figura_thumbnail.png" width="25%" height="25%">

Códigos em R do livro: **Estatística Decodificada** (Silva, A.R. 2021) 

Capítulo 15: **Comparações múltiplas de médias**
____

**Teste t para contrastes**

In [4]:
# exemplo com dados batata.txt
batata <- read.table("https://raw.githubusercontent.com/arsilva87/statsbook/main/datasets/batata.txt", 
     header = TRUE, colClasses = c("factor", "factor", "numeric"))
str(batata)

'data.frame':	12 obs. of  3 variables:
 $ cultivar: Factor w/ 4 levels "1","2","3","4": 1 1 1 2 2 2 3 3 3 4 ...
 $ bloco   : Factor w/ 3 levels "1","2","3": 1 2 3 1 2 3 1 2 3 1 ...
 $ prod    : num  50.9 50.6 51.2 49.1 49.3 49.9 49.9 49.8 49.5 49.2 ...


In [8]:
# ajuste do modelo de ANOVA
lm.batata <- lm(prod ~ cultivar + bloco, data = batata)

In [9]:
# teste do contraste C = m1 + m3 - m2 - m4, com o pacote multcomp
library(multcomp)
c1 <- glht(lm.batata, linfct = mcp(cultivar = c(1, -1, 1, -1)))
summary(c1)


	 Simultaneous Tests for General Linear Hypotheses

Multiple Comparisons of Means: User-defined Contrasts


Fit: lm(formula = prod ~ cultivar + bloco, data = batata)

Linear Hypotheses:
       Estimate Std. Error t value Pr(>|t|)   
1 == 0   1.7667     0.3756   4.703  0.00332 **
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1
(Adjusted p values reported -- single-step method)


In [17]:
# teste do contraste C = m1 + m3 - m2 - m4, com o pacote emmeans
library(emmeans)
lsmeans(lm.batata, specs = "cultivar", contr = list(c1 = c(1, -1, 1, -1)))

$lsmeans
 cultivar lsmean    SE df lower.CL upper.CL
 1          50.9 0.188  6     50.4     51.4
 2          49.4 0.188  6     49.0     49.9
 3          49.7 0.188  6     49.3     50.2
 4          49.4 0.188  6     49.0     49.9

Results are averaged over the levels of: bloco 
Confidence level used: 0.95 

$contrasts
 contrast estimate    SE df t.ratio p.value
 c1           1.77 0.376  6 4.703   0.0033 

Results are averaged over the levels of: bloco 


____
**Teste LSD de Fisher**

In [18]:
# com o pacote ExpDes
library(ExpDes)
with(batata, 
     rbd(treat = cultivar, block = bloco, resp = prod, 
        quali = TRUE, mcomp = "lsd", sigT = 0.05, sigF = 0.05))

------------------------------------------------------------------------
Analysis of Variance Table
------------------------------------------------------------------------
           DF     SS      MS      Fc    Pr>Fc
Treatament  3 4.3825 1.46083 13.8031 0.004218
Block       2 0.4650 0.23250  2.1969 0.192373
Residuals   6 0.6350 0.10583                 
Total      11 5.4825                         
------------------------------------------------------------------------
CV = 0.65 %

------------------------------------------------------------------------
Shapiro-Wilk normality test
p-value:  0.4490271 
According to Shapiro-Wilk normality test at 5% of significance, residuals can be considered normal.
------------------------------------------------------------------------

------------------------------------------------------------------------
Homogeneity of variances test
p-value:  0.2233449 
According to the test of oneillmathews at 5% of significance, the variances can be consider

In [14]:
# com o pacote agricolae
library(agricolae)
with(batata, 
     LSD.test(y = prod, trt = cultivar, DFerror = 6, 
              MSerror = 0.10583, alpha = 0.05, 
              group = FALSE, console = TRUE))

ERROR: Error in library(agricolae): there is no package called ‘agricolae’


____
**A correção de Bonferroni**

In [19]:
with(batata, 
    LSD.test(y = prod, trt = cultivar, DFerror = 6, 
       MSerror = 0.10583, alpha = 0.05, group = FALSE, 
       console = TRUE, p.adj = "bonferroni"))

ERROR: Error in LSD.test(y = prod, trt = cultivar, DFerror = 6, MSerror = 0.10583, : could not find function "LSD.test"


___
**Teste HSD de Tukey**

In [20]:
# quantil da distribuição ou valor tabelado para o exemplo
qtukey(p = 0.95, nmeans = 4, df = 6)

[1] 4.895599

In [21]:
library(agricolae)
with(batata, 
    HSD.test(y = prod, trt = cultivar, DFerror = 6, 
       MSerror = 0.10583, alpha = 0.05, 
       group = FALSE, console = TRUE))

ERROR: Error in library(agricolae): there is no package called ‘agricolae’


___
**Teste SNK**

In [22]:
with(batata, 
    SNK.test(y = prod, trt = cultivar, DFerror = 6, 
       MSerror = 0.10583, alpha = 0.05, 
       group = FALSE, console = TRUE))

ERROR: Error in SNK.test(y = prod, trt = cultivar, DFerror = 6, MSerror = 0.10583, : could not find function "SNK.test"


___
**O critério de Scott-Knott**

In [23]:
library(ScottKnott)
sk <- SK(aov(lm.batata), which = "cultivar")
summary(sk)

 Levels    Means SK(5%)
      1 50.90000      a
      3 49.73333      b
      4 49.43333      b
      2 49.43333      b
